In [137]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf


import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
warnings.filterwarnings("ignore")

In [2]:
df= pd.read_csv('option_train.csv')
df.head()

,Value,S,K,tau,r,BS
0,21.670404,431.623898,420.0,0.341270,0.03013,Under
1,0.125000,427.015526,465.0,0.166667,0.03126,Over
2,20.691244,427.762336,415.0,0.265873,0.03116,Under
3,1.035002,451.711658,460.0,0.063492,0.02972,Over
4,39.553020,446.718974,410.0,0.166667,0.02962,Under


In [3]:
df.isnull().sum()

Value    1
S        1
K        2
tau      1
r        0
BS       0
dtype: int64

In [4]:
df.shape

(1680, 6)

In [5]:
df.describe()

,Value,S,K,tau,r
count,1679.000000,1679.000000,1678.000000,1679.000000,1680.000000
mean,15.072013,440.639445,438.241955,0.437519,0.030235
std,14.036492,13.130436,23.408989,7.057555,0.000557
min,0.125000,0.000000,375.000000,0.003968,0.029510
25%,2.255002,433.863864,420.000000,0.119048,0.029820
50%,11.220588,442.525366,440.000000,0.202381,0.030130
75%,25.745904,447.320414,455.000000,0.285714,0.030540
max,60.149367,455.880619,500.000000,250.000000,0.031880


In [6]:
# drop index 879, 292

df.sort_values("S",ascending=False).tail(5)

,Value,S,K,tau,r,BS
780,3.035000,425.472331,445.0,0.277778,0.03126,Over
928,45.305942,425.472331,385.0,0.277778,0.03126,Under
1526,12.086771,425.472331,425.0,0.277778,0.03126,Under
879,4.125000,0.000000,455.0,0.170635,0.03003,Over
292,8.625000,NaN,NaN,NaN,0.03003,Over


In [7]:
# drop index 12,33

df.sort_values("tau",ascending=False).head()

,Value,S,K,tau,r,BS
12,2.315001,448.688109,470.0,250.000000,0.03013,Over
33,2.565000,445.042240,455.0,146.000000,0.03003,Over
58,2.315000,451.221255,480.0,0.392857,0.03044,Over
1441,48.350072,451.221255,410.0,0.392857,0.03044,Under
904,7.190000,451.221255,465.0,0.392857,0.03044,Over


In [8]:
# 'Value" with missing value: drop index 818

df[df["Value"].isnull()]

,Value,S,K,tau,r,BS
818,NaN,431.284616,NaN,0.230159,0.02972,Over


In [9]:
# Drop index 292

df[df["S"].isnull()]

,Value,S,K,tau,r,BS
292,8.625,NaN,NaN,NaN,0.03003,Over


In [10]:
# Drop index 292, 818

df[df["K"].isnull()]

,Value,S,K,tau,r,BS
292,8.625,NaN,NaN,NaN,0.03003,Over
818,NaN,431.284616,NaN,0.230159,0.02972,Over


In [11]:
#Drop index 292

df[df["tau"].isnull()]

,Value,S,K,tau,r,BS
292,8.625,NaN,NaN,NaN,0.03003,Over


In [12]:
index_list = [879, 292,12,33, 818]
df=df.drop(index=index_list)

In [13]:
df.shape

(1675, 6)

### Linear Regression

In [14]:
# Use " S" as y-variable
result = smf.ols(formula="S ~ Value + K + tau + r", 
                data=df).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      S   R-squared:                       0.614
Model:                            OLS   Adj. R-squared:                  0.613
Method:                 Least Squares   F-statistic:                     664.3
Date:                Sun, 01 May 2022   Prob (F-statistic):               0.00
Time:                        20:36:02   Log-Likelihood:                -4959.9
No. Observations:                1675   AIC:                             9930.
Df Residuals:                    1670   BIC:                             9957.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    323.8819      9.816     32.996      0.0

In [15]:
# Use " tau" as y-variable
result = smf.ols(formula="tau ~ Value + K + S + r", 
                data=df).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    tau   R-squared:                       0.384
Model:                            OLS   Adj. R-squared:                  0.383
Method:                 Least Squares   F-statistic:                     260.3
Date:                Sun, 01 May 2022   Prob (F-statistic):          5.89e-174
Time:                        20:36:02   Log-Likelihood:                 1889.3
No. Observations:                1675   AIC:                            -3769.
Df Residuals:                    1670   BIC:                            -3741.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0006      0.210      4.766      0.0

### Fill in data

In [16]:
df_final= pd.read_csv('option_train.csv')
df_final.head()

,Value,S,K,tau,r,BS
0,21.670404,431.623898,420.0,0.341270,0.03013,Under
1,0.125000,427.015526,465.0,0.166667,0.03126,Over
2,20.691244,427.762336,415.0,0.265873,0.03116,Under
3,1.035002,451.711658,460.0,0.063492,0.02972,Over
4,39.553020,446.718974,410.0,0.166667,0.02962,Under


In [17]:
# Fill in outlier S : record 879
# S= 323.8819 + 0.7819 Value + 0.4875 K -30.3082 tau -3382.9421 r
df_final.loc[879]


Value       4.125
S             0.0
K           455.0
tau      0.170635
r         0.03003
BS           Over
Name: 879, dtype: object

In [18]:
S_fix= 323.8819 + 0.7819 *(4.125) + 0.4875 *(455.0) - 30.3082 *(0.170635) - 3382.9421* (0.03003)
df_final.loc[879, "S"]=S_fix
df_final.loc[879]

Value         4.125
S        442.158347
K             455.0
tau        0.170635
r           0.03003
BS             Over
Name: 879, dtype: object

In [19]:
# Fill in outliear tau: record 12,33 
# tau= 1.0006 +  0.0112 Value + 0.0071 K -0.0085 S -10.4245 r
print(df_final.loc[12])
print(df_final.loc[33])

Value      2.315001
S        448.688109
K             470.0
tau           250.0
r           0.03013
BS             Over
Name: 12, dtype: object
Value        2.565
S        445.04224
K            455.0
tau          146.0
r          0.03003
BS            Over
Name: 33, dtype: object


In [20]:
record_list=[12,33]
for i in record_list:
    tau_fix=1.0006 + 0.0112*df_final.loc[i,"Value"] + 0.0071*df_final.loc[i,"K"] -0.0085*df_final.loc[i,"S"] -10.4245*df_final.loc[i,"r"] 
    df_final.loc[i, "tau"]=tau_fix
    print(df_final.loc[i])

Value      2.315001
S        448.688109
K             470.0
tau        0.235589
r           0.03013
BS             Over
Name: 12, dtype: object
Value        2.565
S        445.04224
K            455.0
tau       0.163921
r          0.03003
BS            Over
Name: 33, dtype: object


In [21]:
df_final.isnull().sum()

Value    1
S        1
K        2
tau      1
r        0
BS       0
dtype: int64

In [22]:
df_final=df_final.dropna()

In [23]:
df_final.shape

(1678, 6)

In [24]:
df_final.sort_values("S",ascending=False).tail(5)

,Value,S,K,tau,r,BS
1147,0.315004,425.472331,470.0,0.277778,0.03126,Over
1577,40.719171,425.472331,390.0,0.277778,0.03126,Under
1365,0.190000,425.472331,475.0,0.277778,0.03126,Over
1526,12.086771,425.472331,425.0,0.277778,0.03126,Under
1281,15.543548,425.472331,420.0,0.277778,0.03126,Under


In [25]:
df_final.sort_values("tau",ascending=False).head(5)

,Value,S,K,tau,r,BS
337,1.720000,451.221255,485.0,0.392857,0.03044,Over
1257,22.391690,451.221255,440.0,0.392857,0.03044,Under
756,39.403943,451.221255,420.0,0.392857,0.03044,Under
1441,48.350072,451.221255,410.0,0.392857,0.03044,Under
1387,43.909506,451.221255,415.0,0.392857,0.03044,Under


In [26]:
df_final.to_csv('df_final.csv')

## Explore the Cleaned Dataset

In [27]:
df = pd.read_csv('df_final.csv',index_col=False)
df.head()

,Unnamed: 0,Value,S,K,tau,r,BS
0,0,21.670404,431.623898,420.0,0.341270,0.03013,Under
1,1,0.125000,427.015526,465.0,0.166667,0.03126,Over
2,2,20.691244,427.762336,415.0,0.265873,0.03116,Under
3,3,1.035002,451.711658,460.0,0.063492,0.02972,Over
4,4,39.553020,446.718974,410.0,0.166667,0.02962,Under


In [28]:
df.drop(columns='Unnamed: 0',inplace=True)

In [29]:
df.isnull().sum()

Value    0
S        0
K        0
tau      0
r        0
BS       0
dtype: int64

In [30]:
df.describe()

,Value,S,K,tau,r
count,1678.000000,1678.000000,1678.000000,1678.000000,1678.000000
mean,15.075855,440.908524,438.241955,0.201886,0.030235
std,14.039793,7.523820,23.408989,0.099754,0.000558
min,0.125000,425.472331,375.000000,0.003968,0.029510
25%,2.255001,433.886762,420.000000,0.119048,0.029820
50%,11.235294,442.579723,440.000000,0.202381,0.030130
75%,25.747434,447.320414,455.000000,0.285714,0.030540
max,60.149367,455.880619,500.000000,0.392857,0.031880


## Encoding

In [31]:
dic = {'Under':0, 'Over':1}

In [32]:
a = df['BS'].map(lambda x:dic[x])
df['binary_BS'] = a

In [33]:
df.head()

,Value,S,K,tau,r,BS,binary_BS
0,21.670404,431.623898,420.0,0.341270,0.03013,Under,0
1,0.125000,427.015526,465.0,0.166667,0.03126,Over,1
2,20.691244,427.762336,415.0,0.265873,0.03116,Under,0
3,1.035002,451.711658,460.0,0.063492,0.02972,Over,1
4,39.553020,446.718974,410.0,0.166667,0.02962,Under,0


## Add new variables

### price_gap

In [34]:
df['price_gap'] = df['K']-df['S']*np.exp(df['tau']*df['r'])

### proportion_sole

In [35]:
df['proportion_sole'] = df['S']/df['K']

### k_today

In [36]:
df['k_today'] = df['K']*np.exp(-df['tau']*df['r'])

### proportion_growth

In [37]:
df['proportion_growth'] = np.log(df['S']/df['K'])

### price_rate

In [38]:
df['price_rate'] = (df['K']-df['S'])/(df['r']*df['tau'])

### time_decay

In [39]:
df['time_decay'] = (df['K']-df['S'])/df['tau']

### overview

In [40]:
df.head()

,Value,S,K,tau,r,BS,binary_BS,price_gap,proportion_sole,k_today,proportion_growth,price_rate,time_decay
0,21.670404,431.623898,420.0,0.341270,0.03013,Under,0,-16.084950,1.027676,415.703494,0.027300,-1130.458854,-34.060725
1,0.125000,427.015526,465.0,0.166667,0.03126,Over,1,35.753918,0.918313,462.583650,-0.085217,7290.685993,227.906844
2,20.691244,427.762336,415.0,0.265873,0.03116,Under,0,-16.320898,1.030753,411.576092,0.030289,-1540.488558,-48.001623
3,1.035002,451.711658,460.0,0.063492,0.02972,Over,1,7.435165,0.981982,459.132806,-0.018182,4392.375191,130.541391
4,39.553020,446.718974,410.0,0.166667,0.02962,Under,0,-38.929729,1.089558,407.980954,0.085773,-7438.009654,-220.313846


## Scaling

In [41]:
df.columns

Index(['Value', 'S', 'K', 'tau', 'r', 'BS', 'binary_BS', 'price_gap',
       'proportion_sole', 'k_today', 'proportion_growth', 'price_rate',
       'time_decay'],
      dtype='object')

In [42]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[['S', 'K', 'tau', 'r', 'price_gap','proportion_sole', 'k_today',  'proportion_growth','price_rate', 'time_decay']] =\
scaler.fit_transform(df[['S', 'K', 'tau', 'r', 'price_gap','proportion_sole', 'k_today', 'proportion_growth','price_rate',  'time_decay']])

In [43]:
df.head()

,Value,S,K,tau,r,BS,binary_BS,price_gap,proportion_sole,k_today,proportion_growth,price_rate,time_decay
0,21.670404,-1.234399,-0.779504,1.397692,-0.188942,Under,0,-0.469989,0.358135,-0.859741,0.380433,0.077115,0.078062
1,0.125000,-1.847086,1.143408,-0.353166,1.838567,Over,1,1.802800,-1.712198,1.169301,-1.764771,1.206214,1.238088
2,20.691244,-1.747797,-0.993160,0.641640,1.659141,Under,0,-0.480334,0.416379,-1.038381,0.437426,0.022139,0.016330
3,1.035002,1.436286,0.929751,-1.387765,-0.924587,Over,1,0.561211,-0.506893,1.019943,-0.486715,0.817611,0.806941
4,39.553020,0.772504,-1.206817,-0.353166,-1.104012,Under,0,-1.471581,1.529623,-1.193983,1.495249,-0.768595,-0.746690


## Spilt into three csvs

In [44]:
df_original = df[['Value', 'S', 'K', 'tau', 'r', 'BS', 'binary_BS']]

In [45]:
df_original.to_csv('df_orginal.csv')

In [46]:
df.to_csv('df_full.csv')

In [47]:
df_avoidRepeat = df[['Value', 'S', 'K', 'tau', 'r', 'BS', 'binary_BS', 'price_gap','proportion_sole', 'k_today', 'price_rate', 'time_decay']]

In [48]:
df_avoidRepeat.to_csv('df_avoidRpeat.csv')

In [49]:
df_original = pd.read_csv('df_orginal.csv')
df_original.drop(columns='Unnamed: 0',inplace=True)
df_original.head()

,Value,S,K,tau,r,BS,binary_BS
0,21.670404,-1.234399,-0.779504,1.397692,-0.188942,Under,0
1,0.125000,-1.847086,1.143408,-0.353166,1.838567,Over,1
2,20.691244,-1.747797,-0.993160,0.641640,1.659141,Under,0
3,1.035002,1.436286,0.929751,-1.387765,-0.924587,Over,1
4,39.553020,0.772504,-1.206817,-0.353166,-1.104012,Under,0


In [50]:
df_full = pd.read_csv('df_full.csv')
df_full.drop(columns='Unnamed: 0',inplace=True)
df_full.head()

,Value,S,K,tau,r,BS,binary_BS,price_gap,proportion_sole,k_today,proportion_growth,price_rate,time_decay
0,21.670404,-1.234399,-0.779504,1.397692,-0.188942,Under,0,-0.469989,0.358135,-0.859741,0.380433,0.077115,0.078062
1,0.125000,-1.847086,1.143408,-0.353166,1.838567,Over,1,1.802800,-1.712198,1.169301,-1.764771,1.206214,1.238088
2,20.691244,-1.747797,-0.993160,0.641640,1.659141,Under,0,-0.480334,0.416379,-1.038381,0.437426,0.022139,0.016330
3,1.035002,1.436286,0.929751,-1.387765,-0.924587,Over,1,0.561211,-0.506893,1.019943,-0.486715,0.817611,0.806941
4,39.553020,0.772504,-1.206817,-0.353166,-1.104012,Under,0,-1.471581,1.529623,-1.193983,1.495249,-0.768595,-0.746690


In [51]:
df_full.columns

Index(['Value', 'S', 'K', 'tau', 'r', 'BS', 'binary_BS', 'price_gap',
       'proportion_sole', 'k_today', 'proportion_growth', 'price_rate',
       'time_decay'],
      dtype='object')

In [52]:
df_avoidRpeat = pd.read_csv('df_avoidRpeat.csv')
df_avoidRpeat.drop(columns='Unnamed: 0',inplace=True)
df_avoidRpeat.head()

,Value,S,K,tau,r,BS,binary_BS,price_gap,proportion_sole,k_today,price_rate,time_decay
0,21.670404,-1.234399,-0.779504,1.397692,-0.188942,Under,0,-0.469989,0.358135,-0.859741,0.077115,0.078062
1,0.125000,-1.847086,1.143408,-0.353166,1.838567,Over,1,1.802800,-1.712198,1.169301,1.206214,1.238088
2,20.691244,-1.747797,-0.993160,0.641640,1.659141,Under,0,-0.480334,0.416379,-1.038381,0.022139,0.016330
3,1.035002,1.436286,0.929751,-1.387765,-0.924587,Over,1,0.561211,-0.506893,1.019943,0.817611,0.806941
4,39.553020,0.772504,-1.206817,-0.353166,-1.104012,Under,0,-1.471581,1.529623,-1.193983,-0.768595,-0.746690


## Lasso and Ridge Regression

In [53]:
df_full_x = df_full[['S', 'K', 'tau', 'r','price_gap','proportion_sole', 'k_today',  'proportion_growth','price_rate', 'time_decay']]
df_full_y = df_full[['binary_BS']]

In [54]:
df_original_x = df_original[['S', 'K', 'tau', 'r']]
df_original_y = df_original[['binary_BS']]

In [55]:
df_avoidRpeat_x = df_avoidRpeat[['S', 'K', 'tau', 'r','price_gap','proportion_sole', 'k_today', 'price_rate', 'time_decay']]
df_avoidRpeat_y = df_avoidRpeat[['binary_BS']]

In [56]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
## for LASSO (0.9404)
for i,j in zip([df_full_x,df_original_x,df_avoidRpeat_x],[df_full_y,df_original_y,df_avoidRpeat_y]):
    print('LETS START')
    for alpha in [0.1,0.5,0.8,1,2]:
        for max_iter in [500,1000,1500,2000]:
            kfolds_regresssion = KFold(n_splits = 10, random_state = 2, shuffle = True) 
            regresssion_model = Lasso(alpha=alpha,max_iter=max_iter)
            r2_model_1_cv = cross_val_score(regresssion_model, i, j, cv=kfolds_regresssion)
            print(f'alpha:{alpha},max_iter:{max_iter}')
            print("Linear Regression: \n")
            print("r squared of 10-folds:",r2_model_1_cv,"(mean r squared:",np.mean(r2_model_1_cv),")")

LETS START
alpha:0.1,max_iter:500
Linear Regression: 

r squared of 10-folds: [0.55778203 0.61252011 0.58080193 0.57375114 0.58108589 0.59060801
 0.62401451 0.60142984 0.55679948 0.57722941] (mean r squared: 0.585602236849539 )
alpha:0.1,max_iter:1000
Linear Regression: 

r squared of 10-folds: [0.55778203 0.61252011 0.58080193 0.57375114 0.58108589 0.59060801
 0.62401451 0.60142984 0.55679948 0.57722941] (mean r squared: 0.585602236849539 )
alpha:0.1,max_iter:1500
Linear Regression: 

r squared of 10-folds: [0.55778203 0.61252011 0.58080193 0.57375114 0.58108589 0.59060801
 0.62401451 0.60142984 0.55679948 0.57722941] (mean r squared: 0.585602236849539 )
alpha:0.1,max_iter:2000
Linear Regression: 

r squared of 10-folds: [0.55778203 0.61252011 0.58080193 0.57375114 0.58108589 0.59060801
 0.62401451 0.60142984 0.55679948 0.57722941] (mean r squared: 0.585602236849539 )
alpha:0.5,max_iter:500
Linear Regression: 

r squared of 10-folds: [-2.96039647e-04 -3.79704080e-02 -2.42934996e-03 -1

alpha:2,max_iter:2000
Linear Regression: 

r squared of 10-folds: [-2.96039647e-04 -3.79704080e-02 -2.42934996e-03 -1.46924329e-05
 -7.21364637e-03 -9.74120893e-03 -3.90699013e-03 -5.07668549e-03
 -1.14654964e-02 -4.70992452e-03] (mean r squared: -0.008282444184250726 )
LETS START
alpha:0.1,max_iter:500
Linear Regression: 

r squared of 10-folds: [0.55778203 0.61252011 0.58080193 0.57375114 0.58108589 0.59060801
 0.62401451 0.60142984 0.55679948 0.57722941] (mean r squared: 0.585602236849539 )
alpha:0.1,max_iter:1000
Linear Regression: 

r squared of 10-folds: [0.55778203 0.61252011 0.58080193 0.57375114 0.58108589 0.59060801
 0.62401451 0.60142984 0.55679948 0.57722941] (mean r squared: 0.585602236849539 )
alpha:0.1,max_iter:1500
Linear Regression: 

r squared of 10-folds: [0.55778203 0.61252011 0.58080193 0.57375114 0.58108589 0.59060801
 0.62401451 0.60142984 0.55679948 0.57722941] (mean r squared: 0.585602236849539 )
alpha:0.1,max_iter:2000
Linear Regression: 

r squared of 10-fold

In [57]:
## for Ridge (0.9899)
for i,j in zip([df_full_x,df_original_x,df_avoidRpeat_x],[df_full_y,df_original_y,df_avoidRpeat_y]):
    print('LETS START')
    for alpha in [0.1,0.5,0.8,1,2]:
        for max_iter in [500,1000,1500,2000]:
            kfolds_regresssion = KFold(n_splits = 10, random_state = 2, shuffle = True) 
            regresssion_model = Ridge(alpha=alpha,max_iter=max_iter)
            r2_model_1_cv = cross_val_score(regresssion_model, i, j, cv=kfolds_regresssion)
            print("Linear Regression: \n")
            print("r squared of 10-folds:",r2_model_1_cv,"(mean r squared:",np.mean(r2_model_1_cv),")")

LETS START
Linear Regression: 

r squared of 10-folds: [0.61460134 0.65394301 0.6560662  0.60819428 0.65058152 0.64694035
 0.654203   0.66707743 0.60020139 0.59237407] (mean r squared: 0.6344182593042016 )
Linear Regression: 

r squared of 10-folds: [0.61460134 0.65394301 0.6560662  0.60819428 0.65058152 0.64694035
 0.654203   0.66707743 0.60020139 0.59237407] (mean r squared: 0.6344182593042016 )
Linear Regression: 

r squared of 10-folds: [0.61460134 0.65394301 0.6560662  0.60819428 0.65058152 0.64694035
 0.654203   0.66707743 0.60020139 0.59237407] (mean r squared: 0.6344182593042016 )
Linear Regression: 

r squared of 10-folds: [0.61460134 0.65394301 0.6560662  0.60819428 0.65058152 0.64694035
 0.654203   0.66707743 0.60020139 0.59237407] (mean r squared: 0.6344182593042016 )
Linear Regression: 

r squared of 10-folds: [0.6130934  0.65479602 0.65451428 0.61134048 0.64922908 0.64357192
 0.65419923 0.66484619 0.5989929  0.59389717] (mean r squared: 0.6338480679382503 )
Linear Regress

Linear Regression: 

r squared of 10-folds: [0.61266979 0.65427267 0.6539603  0.61232663 0.64951668 0.64315444
 0.65413533 0.66460484 0.59840411 0.59380108] (mean r squared: 0.6336845866475327 )
Linear Regression: 

r squared of 10-folds: [0.61266979 0.65427267 0.6539603  0.61232663 0.64951668 0.64315444
 0.65413533 0.66460484 0.59840411 0.59380108] (mean r squared: 0.6336845866475327 )
Linear Regression: 

r squared of 10-folds: [0.61266979 0.65427267 0.6539603  0.61232663 0.64951668 0.64315444
 0.65413533 0.66460484 0.59840411 0.59380108] (mean r squared: 0.6336845866475327 )
Linear Regression: 

r squared of 10-folds: [0.61266979 0.65427267 0.6539603  0.61232663 0.64951668 0.64315444
 0.65413533 0.66460484 0.59840411 0.59380108] (mean r squared: 0.6336845866475327 )
Linear Regression: 

r squared of 10-folds: [0.61254892 0.65557755 0.65395879 0.61218234 0.64869669 0.64228793
 0.65401459 0.66381629 0.59873118 0.59445632] (mean r squared: 0.6336270595602895 )
Linear Regression: 

r sq

In [58]:
## for Elasticnet (really bad)
for i,j in zip([df_full_x,df_original_x,df_avoidRpeat_x],[df_full_y,df_original_y,df_avoidRpeat_y]):
    kfolds_regresssion = KFold(n_splits = 10, random_state = 2, shuffle = True) 
    regresssion_model = ElasticNet()
    r2_model_1_cv = cross_val_score(regresssion_model, i, j, cv=kfolds_regresssion)
    print("Linear Regression: \n")
    print("r squared of 10-folds:",r2_model_1_cv,"(mean r squared:",np.mean(r2_model_1_cv),")")

Linear Regression: 

r squared of 10-folds: [-2.96039647e-04 -3.79704080e-02 -2.42934996e-03 -1.46924329e-05
 -7.21364637e-03 -9.74120893e-03 -3.90699013e-03 -5.07668549e-03
 -1.14654964e-02 -4.70992452e-03] (mean r squared: -0.008282444184250726 )
Linear Regression: 

r squared of 10-folds: [-2.96039647e-04 -3.79704080e-02 -2.42934996e-03 -1.46924329e-05
 -7.21364637e-03 -9.74120893e-03 -3.90699013e-03 -5.07668549e-03
 -1.14654964e-02 -4.70992452e-03] (mean r squared: -0.008282444184250726 )
Linear Regression: 

r squared of 10-folds: [-2.96039647e-04 -3.79704080e-02 -2.42934996e-03 -1.46924329e-05
 -7.21364637e-03 -9.74120893e-03 -3.90699013e-03 -5.07668549e-03
 -1.14654964e-02 -4.70992452e-03] (mean r squared: -0.008282444184250726 )


## Linear Regression

In [59]:
kfolds_regresssion = KFold(n_splits = 10,  random_state = 2, shuffle =True)

In [60]:
LR_model_originial = LinearRegression()
LR_cv_original = cross_val_score(LR_model_originial, df_original[['S', 'K', 'tau', 'r']], df_original['Value'], cv=kfolds_regresssion)

print("Linear Regression: \n")
print("Original dataset: r squared of 10-folds:",LR_cv_original,"\n (mean r squared:",np.mean(LR_cv_original),")")

Linear Regression: 

Original dataset: r squared of 10-folds: [0.92801326 0.88337653 0.90566358 0.90146763 0.919104   0.92238467
 0.91422221 0.92244687 0.89970923 0.90179463] 
 (mean r squared: 0.9098182601788887 )


In [61]:
LR_model_full = LinearRegression()
LR_cv_full = cross_val_score(LR_model_full, df_full.iloc[:,np.r_[1:5,7:13]], df_full['Value'], cv=kfolds_regresssion)

print("Linear Regression: \n")
print("Original dataset: r squared of 10-folds:",LR_cv_full,"\n (mean r squared:",np.mean(LR_cv_full),")")

Linear Regression: 

Original dataset: r squared of 10-folds: [0.99214348 0.99016653 0.99187867 0.99064874 0.99072278 0.99040463
 0.99205373 0.9930463  0.99306532 0.99372141] 
 (mean r squared: 0.9917851617378737 )


In [62]:
LR_model_select = LinearRegression()
LR_cv_select = cross_val_score(LR_model_select, df_avoidRpeat.iloc[:,np.r_[1:5,7:11]], df_avoidRpeat['Value'], cv=kfolds_regresssion)

print("Linear Regression: \n")
print("Original dataset: r squared of 10-folds:",LR_cv_select,"\n (mean r squared:",np.mean(LR_cv_select),")")

Linear Regression: 

Original dataset: r squared of 10-folds: [0.98593983 0.98042229 0.98237339 0.97735452 0.9797993  0.98248928
 0.98225423 0.98674353 0.9867899  0.98552161] 
 (mean r squared: 0.9829687892186014 )


## K-nearest Neighbours Regression

In [63]:
KNN_model_originial = KNeighborsRegressor(n_neighbors=5)
KNN_cv_original = cross_val_score(KNN_model_originial, df_original[['S', 'K', 'tau', 'r']], df_original['Value'], cv=kfolds_regresssion)

print("KNN Regression: \n")
print("R squared of 10-folds:",KNN_cv_original,"\n (mean r squared:",np.mean(KNN_cv_original),")")

KNN Regression: 

R squared of 10-folds: [0.98524372 0.98865384 0.97922618 0.98022722 0.97868604 0.97917861
 0.98429425 0.98085503 0.97824085 0.98537299] 
 (mean r squared: 0.9819978734864169 )


In [64]:
KNN_model_full = KNeighborsRegressor(n_neighbors=6)
KNN_cv_full = cross_val_score(KNN_model_full, df_full.iloc[:,np.r_[1:5,7:13]], df_full['Value'], cv=kfolds_regresssion)

print("KNN Regression: \n")
print("R squared of 10-folds:",KNN_cv_full,"\n (mean r squared:",np.mean(KNN_cv_full),")")

KNN Regression: 

R squared of 10-folds: [0.99109609 0.98826427 0.98858856 0.99198859 0.98314144 0.98441592
 0.9934255  0.98900578 0.98794929 0.9925855 ] 
 (mean r squared: 0.9890460940317165 )


In [65]:
KNN_model_full = KNeighborsRegressor(n_neighbors=6)
KNN_cv_select = cross_val_score(KNN_model_full, df_avoidRpeat.iloc[:,np.r_[1:5,7:11]], df_avoidRpeat['Value'], cv=kfolds_regresssion)

print("KNN Regression: \n")
print("R squared of 10-folds:",KNN_cv_select,"\n (mean r squared:",np.mean(KNN_cv_select),")")



KNN Regression: 

R squared of 10-folds: [0.99200274 0.98974535 0.9857577  0.98964626 0.98072678 0.9861141
 0.99312362 0.98801144 0.98814393 0.99230642] 
 (mean r squared: 0.9885578346792077 )


# Final Model

In [66]:
kfolds_regresssion = KFold(n_splits = 10,  random_state = 2,shuffle =True)

In [67]:
X, y = df_full.iloc[:,np.r_[1:5,7:13]], df_full['Value']

In [68]:
def processSubset(feature_set):
    model = LinearRegression()
    X2 = sm.add_constant(X[list(feature_set)])
    model.fit(X2,y.values.ravel())
    score = cross_val_score(model,X2, y.values.ravel(), cv=kfolds_regresssion)
    return {"R_squared":np.mean(score),"model":dict(zip(feature_set, model.coef_))}

In [69]:
def getBest(k):
    results = []
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))
    models = pd.DataFrame(results)
    best_model = models[models.R_squared==models.R_squared.max()]
    print("Processed ", models.shape[0], "models on", k, "predictors")
    # Return the best model, along with some other useful information about the model
    return best_model

In [139]:
models = pd.DataFrame(columns=["R_squared", "model"])
for i in range(0,11):
    models.loc[i] = getBest(i).iloc[0]

AttributeError: 'GradientBoostingClassifier' object has no attribute 'feature_names_in_'

In [71]:
models

,R_squared,model
0,-0.007597,{}
1,0.891717,{'proportion_sole': 0.0}
2,0.965680,"{'proportion_sole': 0.0, 'proportion_growth': ..."
3,0.991132,"{'price_gap': 0.0, 'proportion_sole': -38.5116..."
4,0.991584,"{'tau': 0.0, 'price_gap': 1.029783564949191, '..."
5,0.991742,"{'S': 0.0, 'tau': 6.8627302158901795, 'proport..."
6,0.991812,"{'tau': 0.0, 'price_gap': 1.252885526695283, '..."
7,0.991830,"{'S': 0.0, 'K': -6.89765999342996, 'r': 21.755..."
8,0.991828,"{'S': 0.0, 'K': -7.077075519099766, 'r': 22.31..."
9,0.991808,"{'S': 0.0, 'K': -13.631287341730598, 'tau': 42..."


In [72]:
final_features = list(models.loc[models['R_squared'].idxmax()]['model'].keys())
final_features

['S',
 'K',
 'r',
 'price_gap',
 'proportion_sole',
 'proportion_growth',
 'price_rate']

In [73]:
LR_model_final = LinearRegression()
LR_cv_final = cross_val_score(LR_model_final, df_full[final_features], df_full['Value'], cv=kfolds_regresssion)

print("Linear Regression: \n")
print("Original dataset: r squared of 10-folds:",LR_cv_final,"\n (mean r squared:",np.mean(LR_cv_final),")")

Linear Regression: 

Original dataset: r squared of 10-folds: [0.99213946 0.99022875 0.99181408 0.99072743 0.99080366 0.99052044
 0.99211077 0.99301199 0.99323662 0.99370361] 
 (mean r squared: 0.991829682211371 )


# Predict testing set

In [74]:
df_test = pd.read_csv('option_test_wolabel.csv')

df_test['price_gap'] = df_test['K']-df_test['S']*np.exp(df_test['tau']*df_test['r'])
df_test['proportion_sole'] = df_test['S']/df_test['K']
df_test['proportion_growth'] = np.log(df_test['S']/df_test['K'])
df_test['price_rate'] = (df_test['K']-df_test['S'])/(df_test['r']*df_test['tau'])


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_test[['S', 'K', 'tau', 'r', 'price_gap','proportion_sole', 'proportion_growth','price_rate']] =\
scaler.fit_transform(df_test[['S', 'K', 'tau', 'r', 'price_gap','proportion_sole', 'proportion_growth','price_rate']])

df_test.head()

,S,K,tau,r,price_gap,proportion_sole,proportion_growth,price_rate
0,-1.244577,0.824029,0.872239,2.301632,1.208221,-1.238708,-1.256997,0.539373
1,-1.111534,-0.858199,-0.203862,2.301632,-0.508466,0.483070,0.504319,-0.062840
2,-1.111534,-0.437642,-0.203862,2.301632,-0.073861,0.033662,0.060206,0.136069
3,-1.244577,-1.068477,0.872239,2.301632,-0.747500,0.670026,0.686039,-0.017035
4,-0.831002,-0.858199,-1.548988,2.301632,-0.519105,0.578638,0.597429,-1.040362


In [75]:
kfolds_regresssion = KFold(n_splits = 10, random_state = 2,shuffle =True)

Final_regression_model = LinearRegression()
Final_regression_model.fit(df_full[final_features], df_full['Value'])

LinearRegression()

In [76]:
# in-sample-R-squared
Final_regression_model.score(df_full[final_features], df_full['Value'])

0.992081007026109

In [77]:
# Predict testing dataset
Value_predict = pd.DataFrame(Final_regression_model.predict(df_test[final_features]),columns=['Value'])
Value_predict

,Value
0,2.522999
1,18.279939
2,11.662388
3,23.993997
4,16.770117
...,...
1115,13.411724
1116,0.163835
1117,1.037297
1118,23.736128


In [78]:
BS_predict = pd.read_csv('prediction_BS.csv')
BS_predict

,BS
0,1
1,0
2,0
3,0
4,0
...,...
1115,0
1116,1
1117,1
1118,0


In [79]:
final_predict = pd.concat([Value_predict,BS_predict],axis=1)

final_predict.to_csv('Group_16_prediction.csv',index=False)
final_predict

,Value,BS
0,2.522999,1
1,18.279939,0
2,11.662388,0
3,23.993997,0
4,16.770117,0
...,...,...
1115,13.411724,0
1116,0.163835,1
1117,1.037297,1
1118,23.736128,0


## Random Forest Classfication


In [80]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

In [81]:
# Cross Validation

kfolds = StratifiedKFold(n_splits = 10, shuffle = True)

data=['df_original', 'df_full', 'df_avoidRepeat']
for i in data:
    rf_model1 = RandomForestClassifier(criterion="entropy", n_estimators=50, max_depth=30, max_features='log2')
    if i =='df_original':
        rf_cv= cross_val_score(rf_model1,df_original.iloc[:,1:5], df_original['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {rf_cv} ")
        print(f"Mean Classification error {i}: {1-np.mean(rf_cv)}")
    elif i== 'df_full':
        rf_cv= cross_val_score(rf_model1,df_full.iloc[:,np.r_[1:5,7:13]], df_full['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {rf_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(rf_cv)}")
    else:
        rf_cv= cross_val_score(rf_model1,df_avoidRepeat.iloc[:,np.r_[1:5,7:12]], df_avoidRepeat['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {rf_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(rf_cv)}")

Accuracies of 10-folds df_original: [0.96428571 0.92857143 0.92261905 0.91071429 0.94047619 0.94047619
 0.94642857 0.91071429 0.91616766 0.94011976] 
Mean Classification error df_original: 0.06794268605645848
Accuracies of 10-folds df_full: [0.94642857 0.89285714 0.88690476 0.95238095 0.92857143 0.93452381
 0.92261905 0.92857143 0.94610778 0.92814371]
Mean Classification error df_full: 0.0732891360136868
Accuracies of 10-folds df_avoidRepeat: [0.93452381 0.94047619 0.92857143 0.92857143 0.93452381 0.89285714
 0.875      0.94047619 0.93413174 0.92215569]
Mean Classification error df_avoidRepeat: 0.07687125748502976


In [82]:

kfolds = StratifiedKFold(n_splits = 10, shuffle = True)

data=['df_original', 'df_full', 'df_avoidRepeat']
for i in data:
    rf_model2 = RandomForestClassifier(criterion="gini", n_estimators=6, max_depth=5, max_features='log2')
    if i =='df_original':
        rf_cv= cross_val_score(rf_model2,df_original.iloc[:,1:5], df_original['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {rf_cv} ")
        print(f"Mean Classification error {i}: {1-np.mean(rf_cv)}")
    elif i== 'df_full':
        rf_cv= cross_val_score(rf_model2,df_full.iloc[:,np.r_[1:5,7:13]], df_full['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {rf_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(rf_cv)}")
    else:
        rf_cv= cross_val_score(rf_model2,df_avoidRepeat.iloc[:,np.r_[1:5,7:12]], df_avoidRepeat['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {rf_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(rf_cv)}")

Accuracies of 10-folds df_original: [0.86904762 0.94047619 0.89285714 0.94642857 0.92261905 0.86904762
 0.91666667 0.93452381 0.90419162 0.93413174] 
Mean Classification error df_original: 0.08700099800399186
Accuracies of 10-folds df_full: [0.89285714 0.96428571 0.91666667 0.89880952 0.94642857 0.91071429
 0.91071429 0.91071429 0.94011976 0.91616766]
Mean Classification error df_full: 0.07925220986598236
Accuracies of 10-folds df_avoidRepeat: [0.91666667 0.9047619  0.92857143 0.91071429 0.94047619 0.91666667
 0.88095238 0.86904762 0.92814371 0.93413174]
Mean Classification error df_avoidRepeat: 0.08698674080410596


## Bayes Classification 

In [83]:
from sklearn.naive_bayes import GaussianNB

kfolds = StratifiedKFold(n_splits = 10, shuffle = True)

data=['df_original', 'df_full', 'df_avoidRepeat']
for i in data:
    bayes_model1 = GaussianNB(var_smoothing=1e-9)
    if i =='df_original':
        bayes_cv= cross_val_score(bayes_model1,df_original.iloc[:,1:5], df_original['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}:{bayes_cv} ")
        print(f"Mean Classification error {i}: {1-np.mean(bayes_cv)}")
    elif i== 'df_full':
        bayes_cv= cross_val_score(bayes_model1,df_full.iloc[:,np.r_[1:5,7:13]], df_full['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}:{bayes_cv} ")
        print(f"Mean Classification error {i}: {1-np.mean(bayes_cv)}")
    else: 
        bayes_cv= cross_val_score(rf_model1,df_avoidRepeat.iloc[:,np.r_[1:5,7:12]], df_avoidRepeat['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {bayes_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(bayes_cv)}")

Accuracies of 10-folds df_original:[0.88095238 0.89880952 0.86904762 0.85119048 0.88095238 0.86904762
 0.86309524 0.91071429 0.91017964 0.86227545] 
Mean Classification error df_original: 0.12037353863701161
Accuracies of 10-folds df_full:[0.92261905 0.89285714 0.93452381 0.9047619  0.94047619 0.85714286
 0.9047619  0.9047619  0.92215569 0.89820359] 
Mean Classification error df_full: 0.09177359566581134
Accuracies of 10-folds df_avoidRepeat: [0.92261905 0.94047619 0.9047619  0.92857143 0.91666667 0.92857143
 0.94047619 0.91071429 0.91017964 0.97005988]
Mean Classification error df_avoidRepeat: 0.07269033361847721


In [84]:

data=['df_original', 'df_full', 'df_avoidRepeat']
for i in data:
    bayes_model2 = GaussianNB(var_smoothing=1e-1)
    if i =='df_original':
        bayes_cv= cross_val_score(bayes_model2,df_original.iloc[:,1:5], df_original['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}:{bayes_cv} ")
        print(f"Mean Classification error {i}: {1-np.mean(bayes_cv)}")
    elif i== 'df_full':
        bayes_cv= cross_val_score(bayes_model2,df_full.iloc[:,np.r_[1:5,7:13]], df_full['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}:{bayes_cv} ")
        print(f"Mean Classification error {i}: {1-np.mean(bayes_cv)}")
    else: 
        bayes_cv= cross_val_score(rf_model2,df_avoidRepeat.iloc[:,np.r_[1:5,7:12]], df_avoidRepeat['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {bayes_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(bayes_cv)}")

Accuracies of 10-folds df_original:[0.875      0.91666667 0.85714286 0.88690476 0.91071429 0.82142857
 0.875      0.88690476 0.91616766 0.84431138] 
Mean Classification error df_original: 0.1209759053321926
Accuracies of 10-folds df_full:[0.9047619  0.94047619 0.88690476 0.89880952 0.92261905 0.91071429
 0.91071429 0.88690476 0.88622754 0.92814371] 
Mean Classification error df_full: 0.09237239806102071
Accuracies of 10-folds df_avoidRepeat: [0.91666667 0.91071429 0.91666667 0.91666667 0.92261905 0.91666667
 0.91666667 0.94642857 0.88023952 0.91616766]
Mean Classification error df_avoidRepeat: 0.0840497576276018


# Decision Tree Classification

In [85]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn import linear_model, tree, ensemble
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score



In [86]:
X = df_original[['S', 'K', 'tau', 'r']]
y = df_original['binary_BS']

In [87]:
kfolds = StratifiedKFold(n_splits = 10, shuffle = True)

data=['df_original', 'df_full', 'df_avoidRepeat']
for i in data:
    dt_model1 = DecisionTreeClassifier(criterion="entropy", splitter = 'best', max_depth=6, random_state=42)
    if i =='df_original':
        dt_cv= cross_val_score(dt_model1,df_original.iloc[:,1:5], df_original['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {dt_cv} ")
        print(f"Mean Classification error {i}: {1-np.mean(dt_cv)}")
    elif i== 'df_full':
        dt_cv= cross_val_score(dt_model1,df_full.iloc[:,np.r_[1:5,7:13]], df_full['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {dt_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(dt_cv)}")
    else:
        dt_cv= cross_val_score(dt_model1,df_avoidRepeat.iloc[:,np.r_[1:5,7:12]], df_avoidRepeat['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {dt_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(dt_cv)}")

Accuracies of 10-folds df_original: [0.91666667 0.9047619  0.89880952 0.88095238 0.91071429 0.89880952
 0.91071429 0.92261905 0.95209581 0.92814371] 
Mean Classification error df_original: 0.08757128599942965
Accuracies of 10-folds df_full: [0.875      0.92261905 0.92857143 0.88690476 0.91666667 0.91071429
 0.95238095 0.875      0.89820359 0.93413174]
Mean Classification error df_full: 0.08998075278015405
Accuracies of 10-folds df_avoidRepeat: [0.92261905 0.91666667 0.88690476 0.92261905 0.88690476 0.91071429
 0.94047619 0.89285714 0.8742515  0.91017964]
Mean Classification error df_avoidRepeat: 0.09358069575135453


In [88]:
data=['df_original', 'df_full', 'df_avoidRepeat']
for i in data:
    dt_model2 = DecisionTreeClassifier(criterion="gini", splitter = 'best', max_depth=15, random_state=42)
    if i =='df_original':
        dt_cv= cross_val_score(dt_model2,df_original.iloc[:,1:5], df_original['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {dt_cv} ")
        print(f"Mean Classification error {i}: {1-np.mean(dt_cv)}")
    elif i== 'df_full':
        dt_cv= cross_val_score(dt_model2,df_full.iloc[:,np.r_[1:5,7:13]], df_full['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {dt_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(dt_cv)}")
    else:
        dt_cv= cross_val_score(dt_model2,df_avoidRepeat.iloc[:,np.r_[1:5,7:12]], df_avoidRepeat['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {dt_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(dt_cv)}")

Accuracies of 10-folds df_original: [0.94642857 0.88690476 0.9047619  0.91071429 0.93452381 0.91666667
 0.9047619  0.92857143 0.91616766 0.88622754] 
Mean Classification error df_original: 0.0864271457085829
Accuracies of 10-folds df_full: [0.86904762 0.86309524 0.92261905 0.92857143 0.91666667 0.92857143
 0.89880952 0.91071429 0.91616766 0.93413174]
Mean Classification error df_full: 0.09116053607071561
Accuracies of 10-folds df_avoidRepeat: [0.92261905 0.91071429 0.91071429 0.91071429 0.93452381 0.9047619
 0.93452381 0.92857143 0.8742515  0.92215569]
Mean Classification error df_avoidRepeat: 0.0846449957228399


## K-Nearest Neighbors (KNN) Classification

In [89]:
kfolds = StratifiedKFold(n_splits = 10, shuffle = True, random_state=42)

data=['df_original', 'df_full', 'df_avoidRepeat']
for i in data:
    KNN_model1 = KNeighborsClassifier(n_neighbors=5)
    if i =='df_original':
        KNN_cv= cross_val_score(KNN_model1,df_original.iloc[:,1:5], df_original['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {KNN_cv} ")
        print(f"Mean Classification error {i}: {1-np.mean(KNN_cv)}")
    elif i== 'df_full':
        KNN_cv= cross_val_score(KNN_model1,df_full.iloc[:,np.r_[1:5,7:13]], df_full['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {KNN_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(KNN_cv)}")
    else:
        KNN_cv= cross_val_score(KNN_model1,df_avoidRepeat.iloc[:,np.r_[1:5,7:12]], df_avoidRepeat['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {KNN_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(KNN_cv)}")

Accuracies of 10-folds df_original: [0.89880952 0.89285714 0.93452381 0.94047619 0.97619048 0.91666667
 0.92261905 0.91071429 0.88622754 0.92215569] 
Mean Classification error df_original: 0.07987596236099215
Accuracies of 10-folds df_full: [0.94047619 0.91666667 0.93452381 0.94047619 0.94047619 0.94642857
 0.91071429 0.93452381 0.89221557 0.91616766]
Mean Classification error df_full: 0.07273310521813525
Accuracies of 10-folds df_avoidRepeat: [0.93452381 0.91071429 0.93452381 0.94047619 0.94047619 0.95238095
 0.91071429 0.92857143 0.89820359 0.92215569]
Mean Classification error df_avoidRepeat: 0.07272597661819213


In [90]:
kfolds = StratifiedKFold(n_splits = 10, shuffle = True, random_state=42)

data=['df_original', 'df_full', 'df_avoidRepeat']
for i in data:
    KNN_model2 = KNeighborsClassifier(n_neighbors=10, algorithm='auto', leaf_size=20)
    if i =='df_original':
        KNN_cv= cross_val_score(KNN_model2,df_original.iloc[:,1:5], df_original['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {KNN_cv} ")
        print(f"Mean Classification error {i}: {1-np.mean(KNN_cv)}")
    elif i== 'df_full':
        KNN_cv= cross_val_score(KNN_model2,df_full.iloc[:,np.r_[1:5,7:13]], df_full['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {KNN_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(KNN_cv)}")
    else:
        KNN_cv= cross_val_score(KNN_model2,df_avoidRepeat.iloc[:,np.r_[1:5,7:12]], df_avoidRepeat['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {KNN_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(KNN_cv)}")

Accuracies of 10-folds df_original: [0.91071429 0.91666667 0.92261905 0.94047619 0.95833333 0.92857143
 0.92261905 0.91666667 0.89221557 0.91616766] 
Mean Classification error df_original: 0.07749500998004
Accuracies of 10-folds df_full: [0.93452381 0.93452381 0.92261905 0.94642857 0.94047619 0.94642857
 0.92261905 0.94047619 0.88023952 0.89820359]
Mean Classification error df_full: 0.07334616481323075
Accuracies of 10-folds df_avoidRepeat: [0.93452381 0.94047619 0.92261905 0.94642857 0.94642857 0.94047619
 0.92261905 0.94047619 0.88622754 0.90419162]
Mean Classification error df_avoidRepeat: 0.07155332192757358


## Logistic Regression

In [91]:
from sklearn.linear_model import LogisticRegression


In [92]:
kfolds=StratifiedKFold(n_splits=10,random_state=1, shuffle=True)

data=['df_original', 'df_full', 'df_avoidRepeat']
for i in data:
    logistic_model1= LogisticRegression(penalty="none",C=1,solver= "newton-cg",intercept_scaling=2,max_iter=1000)
    if i =='df_original':
        loR_cv= cross_val_score(logistic_model1,df_original.iloc[:,1:5], df_original['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {loR_cv} ")
        print(f"Mean Classification error {i}: {1-np.mean(loR_cv)}")
    elif i== 'df_full':
        loR_cv= cross_val_score(logistic_model1,df_full.iloc[:,np.r_[1:5,7:13]], df_full['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {loR_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(loR_cv)}")
    else:
        loR_cv= cross_val_score(logistic_model1,df_avoidRepeat.iloc[:,np.r_[1:5,7:12]], df_avoidRepeat['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {loR_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(loR_cv)}")
        
        

Accuracies of 10-folds df_original: [0.9047619  0.95238095 0.88690476 0.95833333 0.91071429 0.9047619
 0.92857143 0.92261905 0.88622754 0.90419162] 
Mean Classification error df_original: 0.08405332192757342
Accuracies of 10-folds df_full: [0.91666667 0.95238095 0.875      0.96428571 0.94047619 0.91666667
 0.94642857 0.92857143 0.86227545 0.91017964]
Mean Classification error df_full: 0.07870687197034498
Accuracies of 10-folds df_avoidRepeat: [0.92261905 0.95238095 0.89285714 0.96428571 0.93452381 0.9047619
 0.94642857 0.92857143 0.8742515  0.91017964]
Mean Classification error df_avoidRepeat: 0.07691402908468792


In [93]:
kfolds=StratifiedKFold(n_splits=10,random_state=1, shuffle=True)

data=['df_original', 'df_full', 'df_avoidRepeat']
for i in data:
    logistic_model2 = LogisticRegression(penalty="l2",C=2,dual= True,solver= "liblinear",max_iter=10000)
    if i =='df_original':
        loR_cv= cross_val_score(logistic_model2,df_original.iloc[:,1:5], df_original['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {loR_cv} ")
        print(f"Mean Classification error {i}: {1-np.mean(loR_cv)}")
    elif i== 'df_full':
        loR_cv= cross_val_score(logistic_model2,df_full.iloc[:,np.r_[1:5,7:13]], df_full['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {loR_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(loR_cv)}")
    else:
        loR_cv= cross_val_score(logistic_model2,df_avoidRepeat.iloc[:,np.r_[1:5,7:12]], df_avoidRepeat['binary_BS'], cv=kfolds )
        print(f"Accuracies of 10-folds {i}: {loR_cv}")
        print(f"Mean Classification error {i}: {1-np.mean(loR_cv)}")
        

Accuracies of 10-folds df_original: [0.9047619  0.95238095 0.88690476 0.95833333 0.9047619  0.9047619
 0.92857143 0.92261905 0.88622754 0.90419162] 
Mean Classification error df_original: 0.08464856002281151
Accuracies of 10-folds df_full: [0.9047619  0.95238095 0.88690476 0.95833333 0.9047619  0.9047619
 0.92857143 0.92261905 0.88622754 0.90419162]
Mean Classification error df_full: 0.08464856002281151
Accuracies of 10-folds df_avoidRepeat: [0.9047619  0.95238095 0.88690476 0.95833333 0.9047619  0.9047619
 0.92857143 0.92261905 0.88622754 0.90419162]
Mean Classification error df_avoidRepeat: 0.08464856002281151


## Boosted Tree Classification

In [94]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

In [95]:
kfolds = StratifiedKFold(n_splits = 10, shuffle = True)

In [96]:
y = pd.DataFrame(df_original.loc[:,'binary_BS'])
X1 = pd.DataFrame(df_original.iloc[:,1:5])
X2 = df_full.drop(columns = ['Value', 'BS','binary_BS'])
X3 = df_avoidRpeat.drop(columns = ['Value', 'BS', 'binary_BS'])

In [97]:
model = GradientBoostingClassifier()

In [98]:
score1 = cross_val_score(model, X1, y.values.ravel(), scoring='accuracy', cv=kfolds)
print("Boosted Tree: \n")
print("accuracies of 10-folds:",score1,"(mean classification error:",1-np.mean(score1),")")

Boosted Tree: 

accuracies of 10-folds: [0.93452381 0.95238095 0.92857143 0.94047619 0.94047619 0.91071429
 0.93452381 0.91666667 0.94610778 0.94011976] (mean classification error: 0.06554391217564848 )


In [99]:
score2 = cross_val_score(model, X2, y.values.ravel(), scoring='accuracy', cv=kfolds)
print("Boosted Tree: \n")
print("accuracies of 10-folds:",score2,"(mean classification error:",1-np.mean(score2),")")

Boosted Tree: 

accuracies of 10-folds: [0.94642857 0.91666667 0.91666667 0.94642857 0.92857143 0.89880952
 0.92261905 0.91666667 0.93413174 0.94011976] (mean classification error: 0.0732891360136868 )


In [100]:
score3 = cross_val_score(model, X3, y.values.ravel(), scoring='accuracy', cv=kfolds)
print("Boosted Tree: \n")
print("accuracies of 10-folds:",score3,"(mean classification error:",1-np.mean(score3),")")

Boosted Tree: 

accuracies of 10-folds: [0.92857143 0.92261905 0.94047619 0.95238095 0.91071429 0.91071429
 0.96428571 0.89880952 0.88622754 0.91017964] (mean classification error: 0.077502138579983 )


#### Best Subset

In [101]:
def processSubset(feature_set):
    X = sm.add_constant(X2[list(feature_set)])
    model = GradientBoostingClassifier()
    classification = model.fit(X,y.values.ravel()).feature_names_in_
    score = cross_val_score(model,X, y.values.ravel(), scoring='accuracy', cv=kfolds)
    return {"error":1-np.mean(score),"model":classification}

In [102]:
def getBest(k):
    results = []
    for combo in itertools.combinations(X2.columns, k):
        results.append(processSubset(combo))
    models = pd.DataFrame(results)
    best_model = models[models.error==models.error.min()]
    print("Processed ", models.shape[0], "models on", k, "predictors")
    return best_model

In [103]:
models = pd.DataFrame(columns=["error", "model"])
for i in range(0,12):
    models.loc[i] = getBest(i).iloc[0]

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


AttributeError: 'GradientBoostingClassifier' object has no attribute 'feature_names_in_'

In [104]:
models = pd.DataFrame(columns=["error", "model"])
for i in range(0,10):
    models.loc[i] = getBest(i).iloc[0]

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


AttributeError: 'GradientBoostingClassifier' object has no attribute 'feature_names_in_'

In [ ]:
models

#### Hypertune

In [105]:
def bt(x,y,r=0.1,n=100,s=1,c='friedman_mse',d=3,ss=2,sl=1):
    model = GradientBoostingClassifier(learning_rate=r,n_estimators=n,subsample=s,criterion=c,max_depth=d,min_samples_split=ss,min_samples_leaf=sl)
    score = cross_val_score(model,x, y.values.ravel(), scoring='accuracy', cv=kfolds)
    print("Mean classification error:",1-np.mean(score))

In [106]:
for i in [200,250,300,350,400]:
    bt(X1, y,n=i)

Mean classification error: 0.061395067008839455
Mean classification error: 0.06734388366124888
Mean classification error: 0.0637475049900198
Mean classification error: 0.06735101226119189
Mean classification error: 0.06198317650413454


In [107]:
#learning rate
for i in [0.01,0.05,0.1,0.2,0.3]:
    bt(X1, y,r=i,n=200)

Mean classification error: 0.0821998859424008
Mean classification error: 0.0655581693755346
Mean classification error: 0.064952238380382
Mean classification error: 0.06317721699458212
Mean classification error: 0.06614984317080119


In [108]:
#criterion
for i in ['friedman_mse', 'squared_error']:
    bt(X1, y,n=200,c=i)

Mean classification error: 0.06734031936127727
Mean classification error: nan


/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py", line 504, in fit
    n_stages = self._fit_stages(
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py", line 561, in _fit_stages
    raw_predictions = self._fit_stage(
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py", line 214, in _fit_stage
    tree.fit(X, residual, sample_weight=sample_weight,
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 1252, in fit
    super().fit(
  File "/opt/anaconda3/lib/python3.9/site

In [112]:
#max depth
for i in [3,4,5,6,7]:
     bt(X1, y,n=200,d=i)

Mean classification error: 0.07090105503279143
Mean classification error: 0.06614984317080119
Mean classification error: 0.06792486455660085
Mean classification error: 0.07033789563729675
Mean classification error: 0.06911534074707715


In [109]:
#sample split
for i in [2,3,4,6,8,10,30,50]:
    bt(X1, y,n=200,d=5,ss=i)

Mean classification error: 0.07210222412318201
Mean classification error: 0.0655617336755061
Mean classification error: 0.06734388366124888
Mean classification error: 0.0643356144853151
Mean classification error: 0.07032720273738224
Mean classification error: 0.06498431708012542
Mean classification error: 0.06557599087539201
Mean classification error: 0.06614627887082969


In [110]:
#sample leaf
for i in [2,3,4,6,8,10,30,50]:
    bt(X1, y,n=200,ss=4,sl=i)

Mean classification error: 0.060814086113487376
Mean classification error: 0.0643676931850583
Mean classification error: 0.0649486740804106
Mean classification error: 0.06915454804676346
Mean classification error: 0.06734388366124888
Mean classification error: 0.06614984317080119
Mean classification error: 0.06436412888508691
Mean classification error: 0.0655510407755916


In [111]:
##############################################
#X1 FINAL
for i in range (5):
    bt(X1, y,n=200,ss=4,sl=30)

Mean classification error: 0.06556529797547772
Mean classification error: 0.06256059309951512
Mean classification error: 0.06614984317080119
Mean classification error: 0.06914029084687778
Mean classification error: 0.06731893356144847


## Neural Network Classification

In [113]:
model_nn = MLPClassifier(hidden_layer_sizes=(),max_iter=1000)

In [114]:
score1 = cross_val_score(model_nn, X1, y.values.ravel(), scoring='accuracy', cv=kfolds)
print("NN: \n")
print("accuracies of 10-folds:",score1,"(mean classification error:",1-np.mean(score1),")")

NN: 

accuracies of 10-folds: [0.93452381 0.94047619 0.91666667 0.91071429 0.88690476 0.89880952
 0.9047619  0.88690476 0.94610778 0.91616766] (mean classification error: 0.08579626461362966 )


In [117]:
score2 = cross_val_score(model_nn, X2, y.values.ravel(), scoring='accuracy', cv=kfolds)
print("NN: \n")
print("accuracies of 10-folds:",score2,"(mean classification error:",1-np.mean(score2),")")

NN: 

accuracies of 10-folds: [0.92261905 0.93452381 0.91071429 0.91666667 0.92261905 0.92261905
 0.91071429 0.91071429 0.88622754 0.92215569] (mean classification error: 0.08404262902765891 )


In [115]:
scor3 = cross_val_score(model_nn, X3, y.values.ravel(), scoring='accuracy', cv=kfolds)
print("NN: \n")
print("accuracies of 10-folds:",score3,"(mean classification error:",1-np.mean(score3),")")

NN: 

accuracies of 10-folds: [0.92857143 0.92261905 0.94047619 0.95238095 0.91071429 0.91071429
 0.96428571 0.89880952 0.88622754 0.91017964] (mean classification error: 0.077502138579983 )


In [118]:
def nn(x,y,h=(),a='relu',s='adam',al=0.0001,b='auto',l='constant',i=200):
    model = MLPClassifier(hidden_layer_sizes=h,activation=a,solver=s,alpha=al,batch_size=b,learning_rate=l,max_iter=i)
    score = cross_val_score(model_nn, x, y.values.ravel(), scoring='accuracy', cv=kfolds)
    print("Mean classification error:",1-np.mean(score))

In [119]:
#activation
for i in ['identity', 'logistic', 'tanh', 'relu']:
    nn(X3,y,a=i)

Mean classification error: 0.08222483604220121
Mean classification error: 0.08163672654690601
Mean classification error: 0.084630738522954
Mean classification error: 0.08577487881380086


In [120]:
#solver
for i in ['lbfgs', 'sgd', 'adam']:
    nn(X3,y,s=i)

Mean classification error: 0.08347946963216424
Mean classification error: 0.08464143142286873
Mean classification error: 0.08463786712289711


In [122]:
#alpha
for i in [0,0.00005,0.0001,0.0002]:
    nn(X3,y,s='sgd',al=i)

Mean classification error: 0.08582477901340191
Mean classification error: 0.08700099800399208
Mean classification error: 0.08463430282292561
Mean classification error: 0.08581052181351578


In [123]:
#batch
for i in ['auto',50,100,500]:
    nn(X3,y,s='sgd',al=0,b=i)

Mean classification error: 0.08521171941830619
Mean classification error: 0.08637724550898196
Mean classification error: 0.08520459081836318
Mean classification error: 0.08226404334188753


In [124]:
#learning
for i in ['constant','invscaling','adaptive']:
    nn(X3,y,s='sgd',al=0,l=i)

Mean classification error: 0.08282363843741058
Mean classification error: 0.08524379811804972
Mean classification error: 0.08640932420872538


In [125]:
#iter
for j in [500,1000,1500,2000]:
    nn(X3,y,s='sgd',al=0,i=j)

Mean classification error: 0.08402480752780161
Mean classification error: 0.0846236099230111
Mean classification error: 0.0876318790989451
Mean classification error: 0.08341174793270589


In [126]:
#deep
for i in [(),(5,),(5,5),(10,5),(15,10),(15,10,5)]:
    nn(X3,y,h=i,s='sgd',al=0)

Mean classification error: 0.08521528371827769
Mean classification error: 0.0840319361277444
Mean classification error: 0.08341887653264879
Mean classification error: 0.0834188765326489
Mean classification error: 0.08641645280866861
Mean classification error: 0.08519389791844867


In [127]:
#############FINAL#######################
for i in range(5):
    nn(X3,y,h=i,s='sgd',al=0)

Mean classification error: 0.0846164813230681
Mean classification error: 0.08401411462788688
Mean classification error: 0.08344026233247792
Mean classification error: 0.08760692899914457
Mean classification error: 0.08344026233247792


## Prediction

In [128]:
df = pd.read_csv('option_test_wolabel.csv')

In [129]:
df.head()

,S,K,tau,r
0,431.618600,460,0.293651,0.03147
1,432.633296,420,0.182540,0.03147
2,432.633296,430,0.182540,0.03147
3,431.618600,415,0.293651,0.03147
4,434.772855,420,0.043651,0.03147


In [130]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[['S', 'K', 'tau', 'r']] = scaler.fit_transform(df[['S', 'K', 'tau', 'r']])

In [131]:
X = df
X.head()

,S,K,tau,r
0,-1.244577,0.824029,0.872239,2.301632
1,-1.111534,-0.858199,-0.203862,2.301632
2,-1.111534,-0.437642,-0.203862,2.301632
3,-1.244577,-1.068477,0.872239,2.301632
4,-0.831002,-0.858199,-1.548988,2.301632


In [132]:
model = GradientBoostingClassifier(n_estimators=200,min_samples_split=4,min_samples_leaf=30)
model.fit(X1,y.values.ravel())

GradientBoostingClassifier(min_samples_leaf=30, min_samples_split=4,
                           n_estimators=200)

In [133]:
prediction = model.predict(X)

In [134]:
predictions = pd.DataFrame(prediction,columns = ['BS'])

In [135]:
predictions = pd.DataFrame(prediction,columns = ['BS'])

In [136]:
predictions.to_csv('prediction_BS.csv',index=False)